## All possible ways to create a ChoiceDataset

Listed below ! 

In [ ]:
import os
import sys
from pathlib import Path


sys.path.append("../")

import numpy as np
import pandas as pd

from choice_learn.data import ChoiceDataset
from choice_learn.data.storage import FeaturesStorage

We will use the CanadaMode dataset for this example. We can download it directly:

In [ ]:
from choice_learn.datasets import load_modecanada, load_swissmetro

canada_df = load_modecanada(as_frame=True, add_is_public=True)
canada_df.head()

Let's create a column indicating whether the considered transport alternative is individual or not transport.

### From a single dataframe

In [ ]:
dataset = ChoiceDataset.from_single_df(df=canada_df,
                                       fixed_items_features_columns=["is_public"],
                                       contexts_features_columns=["dist", "income", "urban"],
                                       contexts_items_features_columns=["freq", "cost", "ivt", "ovt"],
                                       items_id_column="alt",
                                       contexts_id_column="case",
                                       choices_column="choice",
                                       # the choice columns indicates if the item is chosen (1) or not (0)
                                       choice_mode="one_zero",
                                       )
print(dataset.summary())

Another mode is possible, if the dataframe indicates the name of the chosen item instead of ones and zeros:

In [ ]:
canada_df = load_modecanada(as_frame=True, add_is_public=True, choice_mode="items_id")
canada_df.head()

This time, the choice is not given by ones and zeros but actually names for each context which alternative (item) has been chosen.
The ChoiceDataset handles this case easily, by specifying 'choice_mode="items_id"'.

In [ ]:
dataset = ChoiceDataset.from_single_df(df=canada_df,
                                       fixed_items_features_columns=["is_public"],
                                       contexts_features_columns=["dist", "income", "urban"],
                                       contexts_items_features_columns=["freq", "cost", "ivt", "ovt"],
                                       items_id_column="alt",
                                       contexts_id_column="case",
                                       choices_column="choice",
                                       # the choice columns indicates the id of the chosen item
                                       choice_mode="items_id",
                                       )
print(dataset.summary())

### From several DataFrames

Now, let's say that you have your data split into several files. It can happen if you store the different type of features in different SQL Tables for example.
You will only need to follow some restrictions:

In [ ]:
fixed_items_features, contexts_features, contexts_items_features, choices =\
load_modecanada(as_frame=True, split_features=True, add_is_public=True)

fixed_items_features need to have a column named "item_id" referencing the item. Others columns are free to be any feature.

In [ ]:
fixed_items_features.head()

contexts_features need to have a "context_id" column (otherwise index is used). Other columns are free to be any feature.

In [ ]:
contexts_features.head()

contexts_items_features need to have the column "item_id" and is recommended to have the column "context_id" (otherwise index is used).\
Of course "item_id" and "context_id" should match fixed_items_features and contexts_features.

In [ ]:
contexts_items_features.head()

choices should have a column "context_id" and a column "choice". The value in "choice" should match the values in the column "item_id" in items_features and contexts_items_features.

In [ ]:
choices.head()

In [ ]:
# And now you can create the dataset with:
dataset = ChoiceDataset(fixed_items_features=fixed_items_features,
                        contexts_features=contexts_features,
                        contexts_items_features=contexts_items_features,
                        choices=choices)
print(dataset.summary())

### From several np.ndarrays

Finally, another alternative is to specify each type of feature as np.ndarrays. You can or not also give features names. It is not necessary unless you plan to use a model with specification w.r.t. to those features names.

In [ ]:
fixed_items_features, contexts_features, contexts_items_features, contexts_items_availabilities, choices =\
load_modecanada(split_features=True, add_is_public=True)

If you are using this method, it is your job to make sure that the arrays are well organized.\
First, contexts_features, contexts_items_features, contexts_items_availabilities and choices must be in the right order and their dimension (first one) must match.\
Second, fixed_items_features, contexts_items_availabilities and contexts_items_features must also have the same number of items and ordered the sames. Only it must be on the first dimension of fixed_items_features and the second one of contexts_items_features and contexts_items_availabilities.\
Third, choices must indicate the index of the chosen item as ordered in fixed_items_features and contexts_items_features.
Finally you have to precise the contexts_items_availabilities, or which items were available (1) or not (0) for each context/choice.

To summarize the shape of the arrays must be:
- (n_items, n_fixed_items_features) for fixed_items_features
- (n_choices, n_contexts_features) for contexts_features
- (n_choices, n_items, n_contexts_items_features) for contexts_items_features
- (n_choices, n_items) for contexts_items_availabilities
- (n_choices, ) for choice

*Reminder:* One context corresponds to one choice.

In [ ]:
print("For our example here are the arrays shapes:")
print(f"Fixed Items Features shape: {fixed_items_features.shape}, 4 items, 1 feature (is_public)")
print(f"Contexts Features shape: {contexts_features.shape}, 4324 choices, 3 features (income, dist, urban)")
print(f"Contexts Items Features shape: {contexts_items_features.shape}, 4324 choices, 4 items, 4 features (freq, cost, ivt, ovt)")
print(f"Contexts Items Availabilities shape: {contexts_items_availabilities.shape}, 4324 choices, 4 items")
print(f"Choices shape: {choices.shape}, 4324 choices")

In [ ]:
dataset = ChoiceDataset(fixed_items_features=fixed_items_features,
                        contexts_features=contexts_features,
                        contexts_items_features=contexts_items_features,
                        choices=choices,
                        contexts_items_availabilities=contexts_items_availabilities,
                        # We can give the name of the features as follows, with the right order:
                        fixed_items_features_names=["is_public"],
                        contexts_features_names=["income", "dist", "urban"],
                        contexts_items_features_names=["freq", "cost", "ivt", "ovt"],
                        )
print(dataset.summary())